importing libraries

In [1]:
import praw
import json
import re
import google.generativeai as genai
import time

/home/raichu/Desktop/projects/data_mining/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading crediientials to scrape the reddit

In [8]:
def load_credentials():
    with open("credentials.json") as f:
        data = json.load(f)
    
    reddit = praw.Reddit(
        client_id=data["client_id"],
        client_secret=data["client_secret"],
        user_agent=data["user_agent"],
    )
    
    return reddit

reddit = load_credentials()

script to scrape top post's title and its url and saves in data.json file

In [12]:
subName = "Nepal"
post_limit = None

file_path = 'data.json'

try:
    with open(file_path, 'r') as file:
        data = json.load(file)
except FileNotFoundError:
    data = []
    
submissions = reddit.subreddit(subName).top(time_filter='month',limit=post_limit)
    
for subs in submissions:
    data.append({'id':subs.id,'title':subs.title,'url':subs.url})

with open(file_path, 'w') as file:
    json.dump(data, file, indent=4)
    
print("Data saved to file")
    

loades the data form the data.json file and prints the total number of data

In [3]:
with open('data.json', 'r') as file:
    post_data = json.load(file)

print(len(post_data))

964


this script takes post's url form the data.json and scrapes the posts contents with comments

In [9]:
def scraper(url):
    submission = reddit.submission(url=url)
    body = re.sub(r'\n', ' ', submission.selftext)
    data = []
    comments = []
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        comment = re.sub(r'\n', ' ', comment.body)
        val = re.search("https://", comment)
        if val:
            continue
        else:
            comments.append(comment)
                
    data = {'title': submission.title, 'selftext': body, 'comments': comments}
    return data
    
    

this script saves the data in posts.json file and displys the number of posts scraped

In [13]:
file_path = 'posts.json'

try:
    with open(file_path, 'r') as file:
        data = json.load(file)
except FileNotFoundError:
    data = []
    
submissions = reddit.subreddit(subName).top(time_filter='month',limit=post_limit)
    
for i in range(60):
    post_url = post_data[i]['url']
    data.append(scraper(post_url))

with open(file_path, 'w') as file:
    json.dump(data, file, indent=4)

print(f"{file_path} saved with {len(data)} posts")

posts.json saved with 60 posts


code below loads the credentials for gemini api and loads the model

In [2]:
with open('credentials.json') as f:
    credentials = json.load(f)

GOOGLE_API_KEY=credentials['google_api']
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-pro')

opening the post.json file to get the post body for further analysis

In [6]:
file_path = 'posts.json'
with open(file_path, 'r') as file:
    data = json.load(file)

this script uses the gemini model to determine the intent of the user making the post in the reddit and saves the result in intent.json file

In [ ]:
intent_list = []

for post in data:
    prompt = f"give one word intention of writing the post: {post['selftext']} answer should be among help, frustration, nostalgia, sharing, warning, appereciation, confusion, expose"
    response = model.generate_content(prompt)
    try:
        intent = {'title': post['title'], 'intent': response.text}
    except:
        intent = {'title': post['title'], 'intent': 'None'}
    intent_list.append(intent)
    time.sleep(4)

with open('intent.json', 'w') as f:
    json.dump(intent_list, f, indent=4)

print("generated intent of each post with BERT model successfully!")